In [8]:
from sx_multi import DataSource
from sx_multi import Analysis
from sx_multi import Accumulator
from sx_multi import DaskExecutor

import matplotlib.pyplot as plt

from servicex import ServiceXDataset
from sx_multi import FuncAdlDataset
from coffea import hist, processor

In [9]:
dids = ['mc15_13TeV:mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.merge.DAOD_STDM3.e3601_s2576_s2132_r6630_r6264_p2363_tid05630052_00']
datasets = [
    ServiceXDataset(did, backend_type='xaod')
    for did in dids
]

In [10]:
leptons_per_event_query = FuncAdlDataset() \
        .Select(lambda e: e.Electrons("Electrons")) \
        .Select(lambda eles: eles.Where(lambda e: e.pt()/1000.0 > 30.0)) \
        .Select(lambda eles: eles.Where(lambda e: abs(e.eta()) < 2.5)) \
        .Where(lambda eles: len(eles) == 2) \
        .Select(lambda ls: (ls.Select(lambda e: e.pt()/1000.0), ls.Select(lambda e: e.eta()), ls.Select(lambda e: e.phi()), ls.Select(lambda e: e.m()/1000.0), ls.Select(lambda e: e.charge()))) \
        .AsROOTTTree('data.root', 'mytree', ('electrons_pt', 'electrons_eta', 'electrons_phi', 'electrons_mass', 'electrons_charge'))

In [11]:
datasource = DataSource(query=leptons_per_event_query, metadata={}, datasets=datasets)

In [12]:
class Z_EEAnalysis(Analysis):
    def __init__(self):
        self.accumulator = Accumulator({
            "mass": hist.Hist(
                "Events",
                hist.Bin("mass", "$Z_{ee}$ [GeV]", 60, 60, 120),
            ),
        })

    @staticmethod
    def process(output, events):
        import awkward1 as ak

        dataset = events.metadata['dataset']
        electrons = events.electrons

        # Form the invar mass, plot.
        cut = (ak.num(electrons) == 2)
        diele = electrons[cut][:, 0] + electrons[cut][:, 1]

        output["sumw"][dataset] += len(events)
        output["mass"].fill(
            mass=diele.mass,
        )

        return output

We create the analysis and executor. The `DaskExecutor` can be done two ways:

- `DaskExecutor()` which creates a local cluster. All data will be pulled down to the local machine via an `uproot.open`. This can be paiful depending on what your connection looks like.
- `DaskExecutor(client_addr="node.name.edu:8080")` which will attach to a remote `dask` cluster. This is particularly powerful if the `dask` cluster is located close to the `servicex` installation.

In [13]:
analysis = Z_EEAnalysis()
executor = DaskExecutor()

In [14]:
%matplotlib widget
async def plot_stream(accumulator_stream):
  async for coffea_info in accumulator_stream:
    # Need to ask coffea folks how to anomate this!
    hist.plot1d(coffea_info['mass'])
  return coffea_info

result = await plot_stream(executor.execute(analysis, datasource))

c:\users\gordo\documents\code\iris-hep\servicex-coffea-multiprocessing\.venv\lib\site-packages\distributed\node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 65247 instead
  warnings.warn(


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [53]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
data = np.random.normal(size=100)
fig, axes = plt.subplots()
for _ in range(10):
    plt.hist(data)
    # display(fig)
sleep(10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …